# 2. Getting started

## Loading necessary libraries

In [8]:
import pandas as pd

## Loading JSON dataset and dropping some columns

In [13]:
airbnb = pd.read_json("C:\Github\Data_Science_Portfolio\Airbnb-Price-Prediction\octoparse_airbnb_v1.json")
airbnb = airbnb.drop(columns= ["Keyword", "Host", "roomName", "roomRating", "roomReviewcount"])

airbnb.head(2)

,roomTitle,roomPrice,roomURL,hostType
0,Loft em Campos do Jordão,R$268 por noite,https://www.airbnb.com/rooms/92836566916150808...,Preferido dos hóspedes\nPreferido dos hóspedes
1,Loft em Campos do Jordão,R$99 por noite,https://www.airbnb.com/rooms/53832612?adults=2...,Preferido dos hóspedes\nPreferido dos hóspedes


In [5]:
airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   roomTitle        672 non-null    object
 1   roomRating       672 non-null    object
 2   roomReviewcount  672 non-null    object
 3   roomPrice        672 non-null    object
 4   roomURL          672 non-null    object
 5   hostType         672 non-null    object
dtypes: object(6)
memory usage: 31.6+ KB


In [14]:
airbnb2 = pd.read_json('C:\\Github\\Data_Science_Portfolio\\Airbnb-Price-Prediction\\octoparse_airbnb_v2.json')
airbnb2 = airbnb2.drop(columns= ["Title", "Location", "Number_of_Guests", "Number_of_Bedrooms", "Number_of_Beds", "Number_of_Bath", "Price", "Sleeping_Arrangements", "Hosted_by", "Response_Rate", "Image_1", "Image_2", "Image_3", "Current_Time"])
airbnb2.head(2)

,Page_URL,Rating,Number_of_Reviews,Amenities
0,https://www.airbnb.com/rooms/11369075570005485...,,,Kitchen\nWifi\nFree parking on premises\nHot t...
1,https://www.airbnb.com/rooms/92836566916150808...,4.89,74,


# 3. Joining DataFrames

In [19]:
merged_df = pd.merge(airbnb, airbnb2, left_on='roomURL', right_on='Page_URL')
df = merged_df.drop(columns=["roomURL", "Page_URL"])
print(merged_df.info())
print(merged_df.head(2))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 669 entries, 0 to 668
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   roomTitle          669 non-null    object
 1   roomPrice          669 non-null    object
 2   roomURL            669 non-null    object
 3   hostType           669 non-null    object
 4   Page_URL           669 non-null    object
 5   Rating             669 non-null    object
 6   Number_of_Reviews  669 non-null    object
 7   Amenities          669 non-null    object
dtypes: object(8)
memory usage: 41.9+ KB
None
                  roomTitle        roomPrice  \
0  Loft em Campos do Jordão  R$268 por noite   
1  Loft em Campos do Jordão   R$99 por noite   

                                             roomURL  \
0  https://www.airbnb.com/rooms/92836566916150808...   
1  https://www.airbnb.com/rooms/53832612?adults=2...   

                                         hostType  \
0  Preferido 

# 4. Cleaning

In [23]:
# Renaming columns for consistency
df.rename(columns={
    'roomTitle' : 'roomType',
    'Rating' : 'rating',
    'Number_of_Reviews' : 'numberReviews',
    'Amenities': 'amenities',
}, inplace=True)

In [24]:
# Drop duplicate rows
merged_df.drop_duplicates(inplace=True)

In [25]:
# Find rows with missing values
missing = merged_df[merged_df.isnull().any(axis=1)]

# Display rows with missing values
print(missing)

Empty DataFrame
Columns: [roomTitle, roomPrice, roomURL, hostType, Page_URL, Rating, Number_of_Reviews, Amenities]
Index: []
